In [1]:
# Data Manipulation
import numpy as np
import pandas as pd 

# LAS 
import laspy
import CSF

# Clustering
import hdbscan

In [38]:
file = r'C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS File/cloud.las' # Point Cloud 

In [40]:
las_file = laspy.read(file) # Read LAS File 
points = las_file.points # Access Point Cloud 
points

<ScaleAwarePointRecord(fmt: <PointFormat(2, 0 bytes of extra dims)>, len: 858253, point size: 26)>

In [22]:
point_format = las_file.point_format # Access properties of point clouds
list(point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'red',
 'green',
 'blue']

In [46]:
point_cloud = np.vstack((points.x, points.y, points.z)).transpose() # Extract X, Y, Z and put into a List 

In [47]:
csf = CSF.CSF()

# Parameter Setting
csf.params.bSloopSmooth = False
csf.params.cloth_resolution = 0.50 # more details about parameter: http://ramm.bnu.edu.cn/projects/CSF/download/

In [92]:
csf.setPointCloud(point_cloud)

ground = CSF.VecInt() # a list to indicate the index of ground points after calculation
non_ground = CSF.VecInt() # a list to indicate the index of non-ground points after calculation
csf.do_filtering(ground, non_ground)

In [108]:
outFile = laspy.LasData(las_file.header)
outFile.points = points[np.array(non_ground)] # extract non_ground points, and save it to a las file.

In [109]:
outFile.write(r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS File/out.las")

In [110]:
outPoint_format = outFile.point_format # Access properties of point clouds
list(outPoint_format.dimension_names)

outPoints = outFile.points

outPoint_cloud = np.vstack((outPoints.x, outPoints.y, outPoints.z, outPoints.red, outPoints.blue, outPoints.green)).transpose() # Extract X, Y, Z and put into a List 

In [111]:
# Set-up parameter for HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=100)

In [112]:
# Cluster point clouds using HDBSCAN
labels = clusterer.fit_predict(outPoint_cloud)

In [113]:
# Create a new header for clustered label
header = laspy.LasHeader(point_format=3, version="1.2") 
header.add_extra_dim(laspy.ExtraBytesParams(name="Clusters", type=np.int32))

In [114]:
las = laspy.LasData(header)

las.x = outPoints.x     # x-coordinate
las.y = outPoints.y     # y-coordinate
las.z = outPoints.z     # z-coordinate
las.Clusters = labels   # clustered label
las.write(r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS File/classified.las") # write las file 

In [115]:
del clusterer, labels